<a href="https://colab.research.google.com/github/haruyoshikawabe/accounting-automation-python/blob/main/%E5%8B%98%E5%AE%9A%E7%A7%91%E7%9B%AE%E5%88%A4%E5%AE%9A%E3%83%84%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter
import difflib
from IPython.display import display, HTML
import ipywidgets as widgets
from google.colab import files
import io

class AccountAssistant:
    def __init__(self):
        self.journal_df = None
        self.review_log = []

    def load_journal_csv(self):
        """過去仕訳CSVをアップロード"""
        print("📁 journal.csv をアップロードしてください")
        print("フォーマット: 日付,取引先,摘要,金額,勘定科目")
        uploaded = files.upload()

        for filename in uploaded.keys():
            self.journal_df = pd.read_csv(io.BytesIO(uploaded[filename]))
            # カラム名の正規化
            self.journal_df.columns = self.journal_df.columns.str.strip()
            print(f"✅ {len(self.journal_df)}件の過去仕訳を読み込みました")
            print(f"カラム: {list(self.journal_df.columns)}")
            display(self.journal_df.head())

    def extract_keywords(self, text):
        """キーワード抽出"""
        if pd.isna(text) or text == '':
            return []
        import re
        keywords = re.split(r'[\s、。・]+', str(text))
        return [k for k in keywords if len(k) > 1]

    def calculate_similarity(self, str1, str2):
        """簡易類似度計算"""
        if pd.isna(str1) or pd.isna(str2):
            return 0.0
        return difflib.SequenceMatcher(None, str(str1), str(str2)).ratio()

    def estimate_accounts(self, counterparty, description, amount='', top_k=5):
        """勘定科目候補を推定"""
        if self.journal_df is None:
            print("❌ 先にjournal.csvをアップロードしてください")
            return []

        # カラム名の柔軟な対応
        col_map = {}
        for col in self.journal_df.columns:
            col_lower = col.lower()
            if '取引先' in col or 'counterparty' in col_lower:
                col_map['counterparty'] = col
            elif '摘要' in col or 'description' in col_lower:
                col_map['description'] = col
            elif '勘定科目' in col or 'account' in col_lower:
                col_map['account'] = col

        if 'account' not in col_map:
            print("❌ CSVに勘定科目のカラムが見つかりません")
            return []

        scores = {}
        reasons = {}

        # 過去仕訳を1行ずつ評価
        for _, row in self.journal_df.iterrows():
            account = row[col_map['account']]
            if pd.isna(account) or account == '':
                continue

            if account not in scores:
                scores[account] = 0
                reasons[account] = []

            reason_list = []

            # 1. 取引先一致スコア（最重要: 50点）
            if 'counterparty' in col_map and counterparty:
                row_counterparty = str(row[col_map['counterparty']])
                if row_counterparty.lower() == counterparty.lower():
                    scores[account] += 50
                    reason_list.append(f"✓ 取引先「{row_counterparty}」で過去使用")

            # 2. キーワード一致スコア（15点/語）
            if 'description' in col_map and description:
                row_desc = str(row[col_map['description']])
                input_keywords = self.extract_keywords(description)
                row_keywords = self.extract_keywords(row_desc)

                matched = [kw for kw in input_keywords
                          if any(rk in kw or kw in rk for rk in row_keywords)]

                if matched:
                    scores[account] += len(matched) * 15
                    reason_list.append(f"✓ キーワード一致: {', '.join(matched[:3])}")

            # 3. 摘要類似度（最大20点）
            if 'description' in col_map and description:
                row_desc = str(row[col_map['description']])
                similarity = self.calculate_similarity(row_desc, description)
                if similarity > 0.3:
                    scores[account] += int(similarity * 20)
                    reason_list.append(f"✓ 類似摘要あり（類似度{int(similarity*100)}%）")

            # 4. 頻度ボーナス（1点）
            scores[account] += 1

            if reason_list:
                reasons[account].extend(reason_list)

        # 頻度情報を追加
        for account in scores.keys():
            freq = len(self.journal_df[self.journal_df[col_map['account']] == account])
            if not any('過去使用' in r and '回' in r for r in reasons[account]):
                reasons[account].append(f"✓ 過去使用回数: {freq}回")

        # スコア順にソート
        sorted_accounts = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

        candidates = []
        for account, score in sorted_accounts:
            candidates.append({
                'account': account,
                'score': score,
                'reasons': list(set(reasons[account]))[:3]  # 重複除去＆最大3つ
            })

        return candidates

    def display_candidates(self, candidates):
        """候補を見やすく表示"""
        if not candidates:
            print("⚠️ 候補が見つかりませんでした")
            return

        print("\n" + "="*60)
        print("📊 勘定科目候補（判断根拠付き）")
        print("="*60)

        for i, cand in enumerate(candidates, 1):
            badge = "🥇 最有力" if i == 1 else ""
            print(f"\n{i}. {cand['account']} {badge}")
            print(f"   スコア: {cand['score']}")
            print(f"   判断根拠:")
            for reason in cand['reasons']:
                print(f"      {reason}")
        print("="*60 + "\n")

    def save_review(self, date, counterparty, description, amount,
                   selected_account, memo=''):
        """レビューログに保存"""
        log_entry = {
            '日付': date,
            '取引先': counterparty,
            '摘要': description,
            '金額': amount,
            '選択科目': selected_account,
            'メモ': memo,
            'タイムスタンプ': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        self.review_log.append(log_entry)
        print(f"✅ レビューログに保存しました（累計{len(self.review_log)}件）")

    def export_review_log(self):
        """レビューログをCSV出力"""
        if not self.review_log:
            print("⚠️ 保存されたログがありません")
            return

        df = pd.DataFrame(self.review_log)
        csv_filename = f'review_log_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

        print(f"✅ {csv_filename} として保存しました")
        display(df)
        files.download(csv_filename)

    def show_review_log(self):
        """レビューログを表示"""
        if not self.review_log:
            print("⚠️ まだログがありません")
            return

        df = pd.DataFrame(self.review_log)
        print(f"\n📋 レビューログ（{len(self.review_log)}件）")
        display(df)


# =============================================================================
# 使い方
# =============================================================================

print("""
🎯 勘定科目判断アシスタント
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

過去仕訳をもとに勘定科目候補を推定し、判断根拠を提示します。

【使い方】
1. assistant = AccountAssistant()
2. assistant.load_journal_csv()  # journal.csvをアップロード
3. candidates = assistant.estimate_accounts('取引先名', '摘要', '金額')
4. assistant.display_candidates(candidates)
5. assistant.save_review('2024-01-22', '取引先', '摘要', '5000', '消耗品費', 'メモ')
6. assistant.export_review_log()  # CSV出力

【CSVフォーマット例】
日付,取引先,摘要,金額,勘定科目
2024-01-15,株式会社ABC,事務用品購入,5000,消耗品費
2024-01-16,XYZ商事,接待飲食,8000,交際費
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")

# インスタンス作成
assistant = AccountAssistant()

print("\n✅ 準備完了！上記の使い方を参考に実行してください")
print("最初に assistant.load_journal_csv() を実行してください")


🎯 勘定科目判断アシスタント
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

過去仕訳をもとに勘定科目候補を推定し、判断根拠を提示します。

【使い方】
1. assistant = AccountAssistant()
2. assistant.load_journal_csv()  # journal.csvをアップロード
3. candidates = assistant.estimate_accounts('取引先名', '摘要', '金額')
4. assistant.display_candidates(candidates)
5. assistant.save_review('2024-01-22', '取引先', '摘要', '5000', '消耗品費', 'メモ')
6. assistant.export_review_log()  # CSV出力

【CSVフォーマット例】
日付,取引先,摘要,金額,勘定科目
2024-01-15,株式会社ABC,事務用品購入,5000,消耗品費
2024-01-16,XYZ商事,接待飲食,8000,交際費
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


✅ 準備完了！上記の使い方を参考に実行してください
最初に assistant.load_journal_csv() を実行してください
